In [3]:
from PIL import Image
import numpy as np
import matplotlib.pylab as plt


from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adadelta
from keras.regularizers import l2
from keras.utils import np_utils

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# 1. Read Labels

In [4]:
words = open('words.txt').readlines()
for i in range(len(words)):
    words[i] = words[i][:-1].split('\t')
    
words = dict(words)
    
wnids = open('wnids.txt').readlines()
for i in range(len(wnids)):
    wnids[i] = wnids[i][:-1]

### Now, `words` can be used to look up the image labels

In [5]:
for i in range(len(wnids)):
    print(wnids[i], words[wnids[i]], sep=' ')

n02124075 Egyptian cat
n04067472 reel
n04540053 volleyball
n04099969 rocking chair, rocker
n07749582 lemon
n01641577 bullfrog, Rana catesbeiana
n02802426 basketball
n09246464 cliff, drop, drop-off
n07920052 espresso
n03970156 plunger, plumber's helper
n03891332 parking meter
n02106662 German shepherd, German shepherd dog, German police dog, alsatian
n03201208 dining table, board
n02279972 monarch, monarch butterfly, milkweed butterfly, Danaus plexippus
n02132136 brown bear, bruin, Ursus arctos
n04146614 school bus
n07873807 pizza, pizza pie
n02364673 guinea pig, Cavia cobaya
n04507155 umbrella
n03854065 organ, pipe organ
n03838899 oboe, hautboy, hautbois
n03733131 maypole
n01443537 goldfish, Carassius auratus
n07875152 potpie
n03544143 hourglass
n09428293 seashore, coast, seacoast, sea-coast
n03085013 computer keyboard, keypad
n02437312 Arabian camel, dromedary, Camelus dromedarius
n07614500 ice cream, icecream
n03804744 nail
n04265275 space heater
n02963159 cardigan
n02486410 baboon
n

# 2. Read Images & Labels

#### `labels`: contains the labels that will be used

In [6]:
labels = wnids[:10]

In [7]:
X_train = [] ; Y1_train = [] ; Y2_train = []
X_val = []   ; Y1_val = []   ; Y2_val = []

# create training data
for i in range(len(labels)):

    # open box data
    train_box = open('train/{0}/{0}_boxes.txt'.format(labels[i])).readlines()
    for j in range(len(train_box)):
        train_box[j] = train_box[j][:-1].split('\t')
            
    for k in range(450):

        # add labels & boxes
        # Y1_train.append(labels[i])
        Y1_train.append(i)
        Y2_train.append(list(map(int, train_box[k][1:])))

        # open image
        filename = 'train/{0}/images/{0}_{1}.jpeg'.format(labels[i], k)
        im = np.asarray(Image.open(filename))
        if im.shape == (64,64,3):
            X_train.append(im)
        elif im.shape == (64,64):
            X_train.append(np.dstack([im,im,im]))
        else:
            print('something wrong with {}'.format(filename))

    for l in range(50):
        
        # add labels & boxes
        # Y1_train.append(labels[i])
        Y1_val.append(i)
        Y2_val.append(list(map(int, train_box[l+450][1:])))

        # open image
        filename = 'train/{0}/images/{0}_{1}.jpeg'.format(labels[i], l+450)
        im = np.asarray(Image.open(filename))
        if im.shape == (64,64,3):
            X_val.append(im)
        elif im.shape == (64,64):
            X_val.append(np.dstack([im,im,im]))
        else:
            print('something wrong with {}'.format(filename))

            
X_train = np.stack(X_train, axis=0)
Y1_train = np_utils.to_categorical(Y1_train, 10)
Y2_train = np.stack(Y2_train, axis=0)

X_val = np.stack(X_val, axis=0)
Y1_val = np_utils.to_categorical(Y1_val, 10)
Y2_val = np.stack(Y2_val, axis=0)

In [8]:
# create test data
X_test = []  ; Y1_test = []  ; Y2_test = []

val_annot = open('val/val_annotations.txt').readlines()
for i in range(len(val_annot)):
    val_annot[i] = val_annot[i][:-1].split('\t')
    
for i in range(len(val_annot)):

    if val_annot[i][1] in labels:
        filename = 'val/images/val_{0}.jpeg'.format(i)
        im = np.asarray(Image.open(filename))
        
        # Y1_test.append(val_annot[i][1])
        Y1_test.append(labels.index(val_annot[i][1]))
        Y2_test.append(list(map(int, val_annot[i][2:])))

        if im.shape == (64,64,3):
            X_test.append(im)
        elif im.shape == (64,64):
            X_test.append(np.dstack([im,im,im]))
        else:
            print('something wrong with {}'.format(filename))

X_test = np.stack(X_test, axis=0)
Y1_test = np_utils.to_categorical(Y1_test, 10)
Y2_test = np.stack(Y2_test, axis=0)

In [9]:
X_train.shape, X_val.shape, X_test.shape

((4500, 64, 64, 3), (500, 64, 64, 3), (500, 64, 64, 3))

In [10]:
Y1_train.shape, Y1_val.shape, Y1_test.shape

((4500, 10), (500, 10), (500, 10))

In [11]:
Y2_train.shape, Y2_val.shape, Y2_test.shape

((4500, 4), (500, 4), (500, 4))

# 3. Data Augmentation

# 4. Models

## 4.1. Model 1

In [12]:
model1 = Sequential()
model1.add(Conv2D(4, (3, 3), activation='relu', input_shape=(64, 64, 3), kernel_initializer="glorot_uniform"))
model1.add(Conv2D(4, (3, 3), activation='relu', input_shape=(64, 64, 3), kernel_initializer="glorot_uniform"))
model1.add(Conv2D(4, (3, 3), activation='relu', input_shape=(64, 64, 3), kernel_initializer="glorot_uniform"))
model1.add(MaxPooling2D())
model1.add(Flatten())
model1.add(Dense(10, activation='softmax', kernel_initializer="glorot_uniform"))

np.random.seed(0)
model1.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=Adadelta())

In [13]:
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 62, 62, 4)         112       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 60, 4)         148       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 58, 58, 4)         148       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 29, 29, 4)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3364)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                33650     
Total params: 34,058
Trainable params: 34,058
Non-trainable params: 0
_________________________________________________________________


In [14]:
%%time
hist1 = model1.fit(X_train, Y1_train, epochs=10, batch_size=30, validation_data=(X_val, Y1_val), verbose=2)

Train on 4500 samples, validate on 500 samples
Epoch 1/10
 - 12s - loss: 3.0036 - acc: 0.1189 - val_loss: 2.3145 - val_acc: 0.1220
Epoch 2/10
 - 11s - loss: 2.2273 - acc: 0.1649 - val_loss: 2.3168 - val_acc: 0.1520
Epoch 3/10
 - 11s - loss: 2.0749 - acc: 0.2378 - val_loss: 2.3705 - val_acc: 0.1560
Epoch 4/10
 - 11s - loss: 1.9101 - acc: 0.3131 - val_loss: 2.5104 - val_acc: 0.1560
Epoch 5/10
 - 11s - loss: 1.7551 - acc: 0.3773 - val_loss: 2.7669 - val_acc: 0.1760
Epoch 6/10
 - 11s - loss: 1.6051 - acc: 0.4336 - val_loss: 2.8874 - val_acc: 0.1720
Epoch 7/10
 - 12s - loss: 1.4654 - acc: 0.4816 - val_loss: 3.2344 - val_acc: 0.1720
Epoch 8/10
 - 12s - loss: 1.3370 - acc: 0.5351 - val_loss: 3.4294 - val_acc: 0.1680
Epoch 9/10
 - 11s - loss: 1.2246 - acc: 0.5764 - val_loss: 3.5459 - val_acc: 0.1920
Epoch 10/10
 - 12s - loss: 1.1149 - acc: 0.6129 - val_loss: 3.7518 - val_acc: 0.1600
CPU times: user 4min 24s, sys: 22.3 s, total: 4min 46s
Wall time: 1min 54s


In [ ]:
plt.plot(hist1.history['acc'], 'b-', label="training")
plt.plot(hist1.history['val_acc'], 'r:', label="test")
plt.legend()
plt.show()

In [ ]:
model1.save("mnist_cnn11.hdf5")

In [4]:
import numpy as np
import scipy as sp
import pandas as pd
import sklearn as sk

import matplotlib as mpl
#mpl.use('Agg')
import matplotlib.pylab as plt
from mpl_toolkits.mplot3d import Axes3D

import seaborn as sns
sns.set()
sns.set_style("whitegrid")
sns.set_color_codes()

mpl.rc('figure', figsize=(8, 5))
mpl.rc('figure', dpi=300)

In [1]:
from keras.datasets import mnist
from keras.utils import np_utils

(X_train0, y_train0), (X_test0, y_test0) = mnist.load_data()
X_train = X_train0[:, :, :, np.newaxis].astype('float32') / 255.0
X_test = X_test0[:, :, :, np.newaxis].astype('float32') / 255.0
Y_train = np_utils.to_categorical(y_train0, 10)
Y_test = np_utils.to_categorical(y_test0, 10)

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


11493376/11490434 [==============================] - 1s 0us/step


NameError: name 'np' is not defined